In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np

import DataProcessor

In [2]:
EXPERIMENT_SNS = ['p5972', 'p5973', 'p5974', 'p5975', 'p5979', 'p5993']

In [ ]:
for experiment_SN in EXPERIMENT_SNS:
    
    for event_number in range(9):
        
        if experiment_SN != 'p5993':
            run_number = 2
        else:
            run_number = 1
        
        print(experiment_SN, event_number)
        
        filename = f'../Data/{experiment_SN}e_updated.npz'
        
        data = np.load(filename, allow_pickle = True)
        data = data["experiment"][()]
        event = data['runs'][run_number]['events'][event_number]

        C_f = np.abs(event['rupture_speed'])
        Gamma = np.abs(event['gamma'])
        Xc = np.abs(event['X_c'])
        
        
        
        print(f'Cf: {C_f:.1f}m/s')
        print(f'Gamma: {Gamma:.1f}J/m2')
        print(f'Xc: {Xc:.1f}mm')

p5972 0


KeyError: 'X_c'